In [3]:
import pandas as pd
import numpy as np
import datetime
from filegenerators import *
pd.options.display.float_format = '{:.2e}'.format
from pathlib import Path
from CovCor_calc import OptimaMechtest, OptimaOutput, OptimaSensitivity
import seaborn as sns
import matplotlib.pyplot as plt

In [27]:
class Theoretical_Ranges:
    def __init__(self, range_file: str, sheet_name: str, input_names: list[str],
                 must_be_zero: list[str]) -> None:
        
        self.df_species_ics = pd.read_excel(range_file, sheet_name=sheet_name)
        self.df_species_ics['value'] = self.df_species_ics['value'].astype(float)
        self.must_be_zero = must_be_zero
        self.get_input_data(input_names)
        self.gen_lookuptable()

    def get_input_data(self, input_names: list[str]) -> None:
        self.inputs = {}
        for i in input_names:
            self.inputs[i] = 0.0
        self.inputs["REF"] = 1.0
        self.inputs["Insulin"] = 1e-10

        self.input_data = pd.DataFrame([
            {'species': species, 'minconc': value, 'value': value, 'maxconc': value}
            for species, value in self.inputs.items()])

    def gen_lookuptable(self) -> None:
        self.lut = pd.concat([self.df_species_ics, self.input_data], ignore_index=True)    # look-up-table
        self.lut['species'] = self.lut['species'].str.upper()
        print(f"LUT was created successfully. Its dimensions are: {self.lut.shape}")

    def makeBounds(self, old = True):
        bounds = dict()
        for index, row in self.df_species_ics.iterrows():
            if row.value < 0.1:
                lb = 1e-14
                ub = 1e-13
            elif old:
                lb = (row.value/2)*1e-12
                ub = (row.value*1.5)*1e-12
            else:
                lb = (row.minconc)*1e-12
                ub = (row.maxconc)*1e-12
            bounds[row.species] = [lb, ub]
        return bounds

In [34]:
class Basal:
    def __init__(self, range: Theoretical_Ranges, sampling_type: str, wide = False) -> None:
        self.range = range
        self.wide = wide
        self.old = self.get_sampling_type(sampling_type)   # Should be 'old' or 'new'
        self.bounds = self.range.makeBounds(self.old)
        self.get_observables()
        self.rel_sigmas = self.get_sigma_dict()
        self.get_dataPoints()

    def get_sampling_type(self, sampling_type: str) -> bool:
        if sampling_type == 'old':
            if self.wide == True:
                raise ValueError("If the sampling type is 'old', wide needs to be False.")
            else:
                return True
        elif sampling_type == 'new':
            return False
        else:
            raise ValueError("The sampling type can either be 'old' or 'new'.")

    def __str__(self) -> str:
        kiir = f"Fields:\ndim(bounds): {len(self.bounds)}\n" + \
               f"dim(observables): {len(self.observables)}\n" + \
               f"dim(sigma_dict): {len(self.rel_sigmas)}\n" + \
               f"dim(dataPoints): {self.dataPoints.shape}"
        return kiir

    def get_observables(self) -> None:
        self.observables = []
        for index, row in self.range.df_species_ics.iterrows():
            if row.value > 0:
                self.observables.append(row.species)

    def get_sigma_dict(self) -> dict[str, float]:
        rel_sigmas = dict()

        for index, row in self.range.df_species_ics.iterrows():
            if row.species in self.range.inputs.keys():
                # if row.species not in observables:
                continue
            if row.value in self.range.must_be_zero and row.species not in self.range.inputs.keys():
                rel_sigmas[row.species] = 5e-14
            elif row.species not in self.range.inputs.keys() and row.species in self.observables:
                if self.wide:
                    rel_sigmas[row.species] = ((row.maxconc-row.minconc)/8)*1e-12
                else:
                    rel_sigmas[row.species] = ((row.value*1.5-row.value/2)/8)*1e-12
        return rel_sigmas

    def get_dataPoints(self) -> None:
        columns = list(set(self.observables)-set(self.range.inputs.keys()))
        columns.sort()
        columns.insert(0,'time')
        time = np.linspace(0,24,25)

        self.dataPoints = pd.DataFrame(columns=columns)
        self.dataPoints['time'] = time*60

        # Fill in the "theoretical" stationary conentrations
        for index, row in self.range.df_species_ics.iterrows():
            if row.species in self.dataPoints.columns:
                if row.value == 0:
                    self.dataPoints.loc[:,row.species] = 1e-13
                else:
                    self.dataPoints.loc[:,row.species] = row.value*1e-12

In [43]:
class Genesis:  # As in it creates the xmls ~ the cells {= life ;)} with different initial conditions
    def __init__(self, basal: Basal, num_of_xmls: int,
                 output_directory: str = '/home/nvme/Opt/7_Krisztian/xml/') -> None:
        self.basal = basal
        self.num_xmls = num_of_xmls
        self.maxdigit = len(str(num_of_xmls))
        self.check_output_dir(output_directory)
        self.let_there_be_life()

    def check_output_dir(self, output_directory) -> None:
        if self.basal.old and not self.basal.wide:
            folder_name = f"Basal_{self.num_xmls}_old"
        elif not self.basal.old and not self.basal.wide:
            folder_name = f"Basal_{self.num_xmls}_new"
        else:
            folder_name = f"Basal_{self.num_xmls}_wide_new"

        self.output_dir = os.path.join(output_directory, folder_name)

        if not os.path.exists(self.output_dir):
            os.makedirs(self.output_dir)
            print(f"No existing directory was found at {self.output_dir}. Creating one now...")

    def let_there_be_life(self) -> None:
        species = self.basal.range.df_species_ics.species.to_list()
        inputs = self.basal.range.inputs
        input_names = inputs.keys()
        only_vars = list(set(species)-set(input_names))

        for i in range(1, self.num_xmls+1):
            file_index = i
            generate_file(file_index, self.output_dir, only_vars, inputs,
                          self.basal.bounds, self.basal.dataPoints,
                          self.basal.rel_sigmas, self.maxdigit)

        print("job finished")

In [26]:
gyumik = {'alma': 'finom', 'korte': 'nem rossz', 'meggy': 'nyami', 'eper': 'nyami', 'szilva': 'nyami'}
for i in gyumik.keys():
    print(i)

alma
korte
meggy
eper
szilva


In [44]:
if __name__ == '__main__':
    input_names = ['nS', 'RAP', 'TG', 'dS', 'CCH', 'REF', 'Insulin', 'TG_SERCA', 'mTOR_RAP', 'casp', 'IP3R', 'Baxa', 'tBid']
    must_be_zero = ['casp', 'Baxa', 'tBid', 'p53a', 'PUMA']
    
    range = Theoretical_Ranges('input_files/reactions_ics_finalised.xlsx', 'icranges',
                               input_names, must_be_zero)

    basal = Basal(range, 'old')

    gen = Genesis(basal, 50)

LUT was created successfully. Its dimensions are: (84, 4)


TypeError: 'Theoretical_Ranges' object is not callable